In [1]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
import statsmodels.api as sm

from src.utils import *
from src.data_preprocessing import *
from src.data_processing import *
from src.feature_selection import *
from src.modelisation_arcticle_1 import *

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option("display.max_columns", None)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ismailakrout/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ismailakrout/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# pour ne pas charger le NB avec des warnings 
import warnings 
warnings.filterwarnings('ignore')

## Data PreProcessing

In [3]:
# On lit la base des speechs
# on peut la rendre plus petite pour faire des tests 
df = read_HouseOfCommons(keep_date=False, rd_lines=True, n=10000)

In [4]:
# on ne garde que Labor et Conservative
df = keep_parties(df, ['Lab', 'Con'])

## Data Processing

In [5]:
# On cleen les titres des speechs
df['agenda'] = df['agenda'].apply(clean, args=('unigram',))

In [6]:
# On construit las liste cleen des topics
list_stem_topics = process_list_BigTech_words(topics)

In [7]:
# On filtre le df et on garde uniquement les speechs sur les BigTech
df = keep_Bigtech_speeches(df, list_stem_topics)

In [8]:
df['text'] = df['text'].apply(clean, args=('bigram',))

In [9]:
# On fait le groupby() par speaker et party par la suite pour ne pas dépasser les 1 000 000 carc 
df = df.groupby(by=['Speaker', 'party']).sum().reset_index()

On va repérer les 100 mots (puis 500 quand on aura les speeches en entier) qui sont le plus utilisés par chaque parti, et noter leur fréquence d'apparition. 

In [10]:
df_freqs_Con = count_freqs(df, 'Con')
df_freqs_Lab = count_freqs(df, 'Lab')

On a tous les mots utilisés par le parti Conservateur sur les sujets définis, ainsi que leurs fréquences d'apparition. On va retenir les fréquences avec lesquelles chaques mots sont utilisés, par chaque parti, afin de déterminer le chi2 de chaque mot. On se restreint pour l'instant aux partis Con et Lab, fortement majoritaires. 

In [11]:
df_freqs = merge_freq(df_freqs_Con, df_freqs_Lab)

## feature selection

In [12]:
# On effectue le etst du Qi 2 
df_freqs = test_qi_2(df_freqs)

On a donc ici chaque mot utilisé par les parti sur le sujet délimité, avec le chi2 correspondant. On va se limiter aux 500 premiers mots :

In [13]:
df_freqs = keep_significatif_word(df_freqs, n=500)

## Data Processing Bis

In [14]:
list_of_words = selected_words(df_freqs)

In [15]:
df = construct_df_reg(df, df_freqs, list_of_words)

In [16]:
df = normalize(df, list_of_words)

In [17]:
df = party_str_to_dummy(df)

## Regressions linéaires (Modélisation)

In [18]:
# rajout des lignes slopes_coeff et intercept

df = regress_df(df, list_of_words)

In [19]:
# df.to_csv('01. output/df_500_with_slope_coef_bis.csv')